# Authentication to Pods Hosted on Tapis

This notebook is how we authenticate our Neo4j database to TACC! We do this using tapis, and more specifically the pods API. 
Pods are a tool which allow for simple creation of a Neo4j DBMS hosted at TACC simply through some basic commands.
At a high level, we create a Tapis object associated with an individual account. Then, a pod is created under that username, with the Neo4j template. 
This pod has an associated username and password to connect to the DBMS. A simple command reveals this information. It is also straightforward to add in other users to have
admin access to this pod. The pod has an associated bolt url, which can be connected to via a Neo4j API like py2neo in a jupyter notebook. It can also be connected to via the 
Neo4j browser. And that's it! The Neo4j database is hosted and requires authentication to TACC via Tapis!

In [ ]:
import time
import pprint as pp
import datetime
from getpass import getpass
import pytz
import neo4jupyter
from py2neo import Graph
from tapipy.tapis import Tapis

In [ ]:
def show(res):
    try:
        pp.pprint(res.json())
    except:
        pp.pprint(res.text)

neo4jupyter.init_notebook_mode()

In [ ]:
start = time.time()

# Base URL for Tapis
base_url = "https://icicle.tapis.io"
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

In [ ]:
# Returns all of the pods registered to the account. 
t.pods.get_pods()

In [ ]:
# Used to reference the pod
pod_id = str(input("Enter a pod ID ")).lower()

In [ ]:
# Use neo4j to create a pod for a neo4j DBMS
pod_template = str(input("enter a pod template (enter neo4j)"))

In [ ]:
# Describe the pod
pod_description = str(input("enter a pod description"))

In [ ]:
# Run this to create the pod
t.pods.create_pod(pod_id=pod_id, pod_template=pod_template, description=pod_description)

In [ ]:
# Setting permissions 
t.pods.set_pod_permission(pod_id=pod_id, user="SillyFellow", level="ADMIN")

In [15]:
# Setting username and password for authentication
username, password = t.pods.get_pod_credentials(pod_id=pod_id).user_username, t.pods.get_pod_credentials(pod_id=pod_id).user_password

In [ ]:
# Please remember that it is never a good idea to print out your passwords! Very dangerous!

In [20]:
# Testing connection to the bolt service
graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.tapis.io:443", auth=(username, password), secure=True, verify=True)

In [21]:
# Further testing connection via query
query = "create (:N)-[:R]->(:V)"
graph.run(query).to_data_frame()

""


In [22]:
# Test visualization of test nodes
neo4jupyter.draw(graph, {"Organization" : "alias"})

In [ ]:
t.pods.delete_pod(pod_id=pod_id)